In [1]:
% Add SPM to the path. In a matlab prompt, you can type this to find the answer --> spm('dir')
addpath('/home/remi/matlab/SPM/spm12');

% we add all the subfunctions that are in the sub directories
addpath(genpath(fullfile(pwd, '..', 'src')));
addpath(genpath(fullfile(pwd, '..', 'lib')));

checkDependencies();

___________________________________________________________________________
___________________________________________________________________________
                                                   
                 __  ____  ____     _      _    _  
                / _)(  _ \(  _ \   | |    / \  | ) 
               ( (_  )___/ )___/   | |_  / _ \ | \ 
                \__)(__)  (__)     |___||_/ \_||__)
                                                   
Thank you for using the CPP lap pipeline - version v0.1.0. 

Current list of contributors includes
 Mohamed Rezk
 Rémi Gau
 Olivier Collignon
 Ane Gurtubay
 Marco Barilari
 Ceren Battal


Please cite using the following DOI: 
 https://doi.org/10.5281/zenodo.3554331.

For bug report & suggestions see our github repo: 
 https://github.com/cpp-lln-lab/CPP_BIDS_SPM_pipeline

___________________________________________________________________________
___________________________________________________________________________


Checkin

In [6]:
% task to analyze
opt.taskName = 'auditory';

% Uncomment the line below to run preprocessing in "native" space.
% - use realign and unwarp
% - don't do normalization
% opt.space = 'individual';

% The directory where the data are located
opt.dataDir = fullfile(pwd, 'output', 'MoAEpilot');

opt.model.file = fullfile('..', 'demos', 'MoAEpilot', 'models', 'model-MoAE_smdl.json');

% specify the result to compute
opt.result.Steps(1) = struct( ...
                           'Level',  'subject', ...
                           'Contrasts', struct( ...
                                               'Name', 'listening', ... % has to match
                                               'Mask', false, ...
                                               'MC', 'FWE', ... FWE, none, FDR
                                               'p', 0.05, ...
                                               'k', 0, ...
                                               'NIDM', true));

opt.result.Steps(1).Contrasts(2) = struct( ...
                                        'Name', 'listening_inf_baseline', ...
                                        'Mask', false, ...
                                        'MC', 'none', ... FWE, none, FDR
                                        'p', 0.01, ...
                                        'k', 0, ...
                                        'NIDM', true);

opt = checkOptions(opt);
saveOptions(opt);

Options saved in: 



In [3]:
% URL of the data set to download
URL = 'http://www.fil.ion.ucl.ac.uk/spm/download/data/MoAEpilot/MoAEpilot.bids.zip';

%% Get data
fprintf('%-40s:', 'Downloading dataset...');
urlwrite(URL, 'MoAEpilot.zip');
unzip('MoAEpilot.zip', fullfile(pwd, 'output'));

In [4]:
!tree output/

output/
└── MoAEpilot
    ├── CHANGES
    ├── dataset_description.json
    ├── README
    ├── sub-01
    │   ├── anat
    │   │   └── sub-01_T1w.nii
    │   └── func
    │       ├── sub-01_task-auditory_bold.nii
    │       └── sub-01_task-auditory_events.tsv
    └── task-auditory_bold.json

4 directories, 7 files



In [12]:
% TODO
% reportBIDS(opt)


-------------------------
  Reading BIDS: /home/remi/github/CPP_BIDS_SPM_pipeline/notebooks/output/MoAEpilot
-------------------------
Done.

Working on anat...
 - /home/remi/github/CPP_BIDS_SPM_pipeline/notebooks/output/MoAEpilot/sub-01/anat/sub-01_T1w.nii
  Opening file /home/remi/github/CPP_BIDS_SPM_pipeline/notebooks/output/MoAEpilot/sub-01/anat/sub-01_T1w.nii.

 ANAT REPORT 
T1w [XXXX] [XXXX] structural MRI data were collected (54 slices; 
repetition time, TR= [XXXX] ms; echo time, TE= [XXXX] ms; flip angle, FA=[XXXX] deg; 
field of view, FOV= 256.00 X 256.00 mm; matrix size= 256 X 256; voxel size= 1.00 X 1.00 X 3.00 mm) 


Working on func...
 - /home/remi/github/CPP_BIDS_SPM_pipeline/notebooks/output/MoAEpilot/sub-01/func/sub-01_task-auditory_bold.nii
  Opening file /home/remi/github/CPP_BIDS_SPM_pipeline/notebooks/output/MoAEpilot/sub-01/func/sub-01_task-auditory_bold.nii.

 FUNC REPORT 
1 run(s) of auditory [XXXX] [XXXX] fMRI data were collected (64 slices acquired in a [XXXX]

In [10]:
bidsCopyRawFolder(opt, 1);


Options are locked & loaded.

derivatives directory created: /home/remi/github/CPP_BIDS_SPM_pipeline/notebooks/output/MoAEpilot/../derivatives/SPM12_CPPL 
 json files copied to derivatives directory 
FOR TASK: auditory
WILL WORK ON SUBJECTS

{
  [1,1] = 01
}
folder copied: sub-01 


In [11]:
!tree output/

output/
├── derivatives
│   └── SPM12_CPPL
│       ├── dataset_description.json
│       ├── sub-01
│       │   ├── anat
│       │   │   └── sub-01_T1w.nii
│       │   └── func
│       │       ├── sub-01_task-auditory_bold.nii
│       │       └── sub-01_task-auditory_events.tsv
│       └── task-auditory_bold.json
└── MoAEpilot
    ├── CHANGES
    ├── dataset_description.json
    ├── README
    ├── sub-01
    │   ├── anat
    │   │   └── sub-01_T1w.nii
    │   └── func
    │       ├── sub-01_task-auditory_bold.nii
    │       └── sub-01_task-auditory_events.tsv
    └── task-auditory_bold.json

9 directories, 12 files



In [13]:
% TODO
% bidsSTC(opt)


Options are locked & loaded.

FOR TASK: auditory
WILL WORK ON SUBJECTS

{
  [1,1] = 01
}
DOING SLICE TIME CORRECTION
 PROCESSING GROUP: SUBJECT No.: 1 SUBJECT ID : 01 
 SLICE TIMING INFORMATION COULD NOT BE EXTRACTED FROM METADATA.
 CHECKING IF SPECIFIED IN opt IN THE "getOption" FUNCTION.

 SLICE TIMING INFORMATION COULD NOT BE EXTRACTED.
    getSliceOrder at line 50 column 7
    setBatchSTC at line 23 column 14
    bidsSTC at line 51 column 19
    setBatchSTC at line 25 column 5
    bidsSTC at line 51 column 19


In [14]:
% Be patient this one takes a while
bidsSpatialPrepro(opt);

% The following do not run on octave for now (because of spmup)
% anatomicalQA(opt);
% bidsResliceTpmToFunc(opt);
% functionalQA(opt);



Options are locked & loaded.

FOR TASK: auditory
WILL WORK ON SUBJECTS

{
  [1,1] = 01
}
DOING SPATIAL PREPROCESSING
 PROCESSING GROUP: SUBJECT No.: 1 SUBJECT ID : 01 
 BUILDING SPATIAL JOB : SELECTING ANATOMCAL
 BUILDING SPATIAL JOB : REALIGN
 /home/remi/github/CPP_BIDS_SPM_pipeline/notebooks/output/derivatives/SPM12_CPPL/sub-01/func/sub-01_task-auditory_bold.nii
 BUILDING SPATIAL JOB : COREGISTER
 BUILDING SPATIAL JOB : SEGMENT ANATOMICAL
 BUILDING SPATIAL JOB : SKULL STRIPPING
 BUILDING SPATIAL JOB : NORMALIZE FUNCTIONALS
 BUILDING SPATIAL JOB : NORMALIZE FUNCIONAL
 BUILDING SPATIAL JOB : NORMALIZE STRUCTURAL
 BUILDING SPATIAL JOB : NORMALIZE GREY MATTER
 BUILDING SPATIAL JOB : NORMALIZE WHITE MATTER
 BUILDING SPATIAL JOB : NORMALIZE CSF
Initialising batch system... mv: cannot move '/home/remi/matlab/SPM/spm12/toolbox/.' to '/home/remi/matlab/SPM/spm12/toolbox/MACS/.': Device or resource busy
done.


------------------------------------------------------------------------
01-Nov-20

In file "/home/remi/matlab/SPM/spm12/@file_array/subsref.m" (v7439), function "subsref" at line 65.
In file "/home/remi/matlab/SPM/spm12/@nifti/subsref.m" (v7147), function "subsref>rec" at line 221.
In file "/home/remi/matlab/SPM/spm12/@nifti/subsref.m" (v7147), function "subsref" at line 45.
In file "/home/remi/matlab/SPM/spm12/spm_deformations.m" (v6577), function "spm_deformations>get_def" at line 192.
In file "/home/remi/matlab/SPM/spm12/spm_deformations.m" (v6577), function "spm_deformations>get_job" at line 78.
In file "/home/remi/matlab/SPM/spm12/spm_deformations.m" (v6577), function "spm_deformations>get_comp" at line 48.
In file "/home/remi/matlab/SPM/spm12/spm_deformations.m" (v6577), function "spm_deformations" at line 17.
In file "/home/remi/matlab/SPM/spm12/config/spm_run_norm.m" (v7406), function "spm_run_norm>write_norm" at line 111.
In file "/home/remi/matlab/SPM/spm12/config/spm_run_norm.m" (v7406), function "spm_run_norm" at line 29.

01-Nov-2020 08:56:57 - Running '

In [ ]:
% Smoothing to apply
FWHM = 6;
bidsSmoothing(FWHM, opt);

In [ ]:
bidsFFX('specifyAndEstimate', opt, FWHM);
bidsFFX('contrasts', opt, FWHM);
bidsResults(opt, FWHM);